In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import webcolors

In [ ]:
df = pd.read_csv(r"C:\Users\sowmy\Downloads\fashion-mnist_train.csv.zip")


y = df.iloc[:, 0].values 
X = df.iloc[:, 1:].values 

X = X / 255.0  

X = X.reshape(-1, 28, 28, 1)


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
sns.set_style("darkgrid")


plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(X_train[i].reshape(28, 28), cmap='gray')
    plt.title(class_names[y_train[i]])
    plt.axis('off')
plt.suptitle("Sample Images from Fashion-MNIST Dataset", fontsize=14)
plt.show()


plt.figure(figsize=(8, 5))
sns.countplot(x=y_train, palette="viridis")
plt.xticks(ticks=range(10), labels=class_names, rotation=45)
plt.xlabel("Clothing Category")
plt.ylabel("Frequency")
plt.title("Class Distribution in Fashion-MNIST")
plt.show()

plt.figure(figsize=(8, 5))
sns.histplot(X_train.flatten(), bins=50, kde=True, color='blue')
plt.xlabel("Pixel Intensity")
plt.ylabel("Count")
plt.title("Pixel Intensity Distribution")
plt.show()

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(28, 28, 1)), 
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
feature_extractor = keras.Model(inputs=model.input, outputs=model.layers[-3].output)
image_embeddings = feature_extractor.predict(X_test)

def find_similar_images(query_image, num_results=5):
    query_embedding = feature_extractor.predict(query_image.reshape(1, 28, 28, 1))
    similarities = cosine_similarity(query_embedding, image_embeddings)
    top_indices = np.argsort(similarities[0])[-num_results:][::-1]

    plt.figure(figsize=(10, 2))
    for i, idx in enumerate(top_indices):
        plt.subplot(1, num_results, i+1)
        plt.imshow(X_test[idx].reshape(28, 28), cmap='gray')
        plt.title(class_names[y_test[idx]])
        plt.axis('off')
    plt.show()

find_similar_images(X_test[10])


In [ ]:
def text_based_search(query):
    matches = [i for i, name in enumerate(class_names) if query.lower() in name.lower()]
    if matches:
        print(f"Found {len(matches)} results:")
        for idx in matches:
            plt.figure(figsize=(1.5, 1.5))
            plt.imshow(X_test[idx].reshape(28, 28), cmap='gray')
            plt.title(class_names[y_test[idx]])
            plt.axis('off')
            plt.show()
    else:
        print("No matches found!")

text_based_search("shirt")

In [ ]:
def closest_color(requested_color):
    min_colors = {}
    for hex_code, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(hex_code)
        min_colors[(r_c, g_c, b_c)] = name
    closest = min(min_colors.keys(), key=lambda color: np.linalg.norm(np.array(color) - np.array(requested_color)))
    return min_colors[closest]

def hex_color_based_search(hex_code):
    requested_rgb = np.array(webcolors.hex_to_rgb(hex_code))
    matching_indices = []
    
    for i, img in enumerate(X_test):
        avg_color = np.mean(img) * 255 
        avg_rgb = (avg_color, avg_color, avg_color) 
        color_diff = np.linalg.norm(np.array(avg_rgb) - requested_rgb)

        if color_diff < 50: 
            matching_indices.append(i)

    if matching_indices:
        print(f"Found {len(matching_indices)} matching items for color {hex_code}:")
        plt.figure(figsize=(10, 2))
        for i, idx in enumerate(matching_indices[:5]):
            plt.subplot(1, 5, i+1)
            plt.imshow(X_test[idx].reshape(28, 28), cmap='gray')
            plt.title(class_names[y_test[idx]])
            plt.axis('off')
        plt.show()
    else:
        print("No matching colors found.")


hex_color_based_search("#808080")